In [ ]:
import requests

import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [ ]:
url = 'https://tamanhhospital.vn/chu-de-tu-van/tu-van/page/{page_num}/'
page_num = 0
hrefs_list = list()

while True:
    page_num += 1
    response = requests.get(url.format(page_num=page_num))
    soup = BeautifulSoup(response.text, 'html.parser')
    a_tags = soup.find_all('a', class_='cl_33 small')

    if len(a_tags) == 0:
        break

    for tag in tqdm(a_tags, desc=f"Page {page_num}"):
        hrefs_list.append(tag.get('href'))

Page 366: 100%|██████████| 10/10 [00:00<00:00, 13323.71it/s]


In [ ]:
with open("urls.txt", 'w') as file:
    for url in hrefs_list:
        file.write(url + '\n')

In [ ]:
with open("urls.txt", 'r') as file:
    hrefs_list = file.read().splitlines()

In [ ]:
hrefs_list[:3]

['https://tamanhhospital.vn/tu-van/viem-tuyen-giap-co-phai-mo-khong/',
 'https://tamanhhospital.vn/tu-van/tre-bi-viem-mang-nao-co-chua-duoc-khong/',
 'https://tamanhhospital.vn/tu-van/lang-lang-dung-khong-vung-la-bi-gi/']

In [ ]:
def validate_child(child):
    ban_classes = ["alert", "item-info", "mb_10", "wp-caption"]
    if child.get("class", False) != False:
        validated_classes = [cls in ban_classes for cls in child["class"]]
        if sum(validated_classes) > 0:
            return False

    if len(child.text):
        return True

    return False

In [ ]:
def get_content(content_tag):
    child_texts = list()
    for child in content_tag.find_all(recursive=False):
        if validate_child(child) == False:
             if (child.get("class", False) != False and
                 "alert" in child["class"] and
                 "Để được tư vấn" in child_texts[-1]):
                child_texts.pop(-1)
        else:
            child_texts.append(child.text)

    return "\n".join(child_texts)

In [ ]:
def href2dict(href):
    try:
        response = requests.get(href)
        soup = BeautifulSoup(response.text, 'html.parser')

        # print("#" * 79)
        # print(f"{href = }")

        title = soup.find('div', class_='title').text.strip()
        # print(f"{title = }")

        description = soup.find_all('div', class_="mt_10")[-1].text
        # print(f"{description = }")

        content_tag = soup.find('div', class_='tuvan_detail')
        content = get_content(content_tag)
        # print("content = \n", content)

        return {
            "title": title.replace("\u200b", "").replace("\u00a0", ""),
            "description": description.replace("\u200b", "").replace("\u00a0", ""),
            "content": content.replace("\u200b", "").replace("\u00a0", ""),
        }
    except Exception as e:
        print(e)
        print(href)

In [ ]:
# a = href2dict('https://tamanhhospital.vn/tu-van/tre-bi-viem-mang-nao-co-chua-duoc-khong/')
# a = href2dict("https://tamanhhospital.vn/tu-van/viem-tuyen-giap-co-phai-mo-khong/")
# a = href2dict("https://tamanhhospital.vn/tu-van/tre-bi-tay-chan-mieng-sot-may-ngay/")

In [ ]:
import concurrent.futures

num_workers = 6
# df_list = [pd.DataFrame(columns=['title', 'description', 'content']) for __ in range(num_workers)]
# url_partitions = split_list_into_partitions(hrefs_list, num_workers)

with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    results = list(tqdm(executor.map(href2dict, hrefs_list), total=len(hrefs_list)))

100%|██████████| 3660/3660 [12:01<00:00,  5.08it/s]


In [ ]:
len(results)

3660

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df.head()

,title,description,content
0,Viêm tuyến giáp có phải mổ không? Khi nào nên ...,"Chào bác sĩ, 6 tháng trước, mẹ em bị sưng, đau...",Bệnh viêm tuyến giáp không chỉ ảnh hưởng đến s...
1,Trẻ bị viêm màng não có chữa được không? Ba mẹ...,Dạo gần đây thời tiết nắng nóng nên em thường ...,Trẻ bị viêm màng não có chữa được không? Viêm ...
2,Lâng lâng đứng không vững là bị gì?,E chào bác sĩ cho em hỏi em hay bị lang lang n...,Chào bạn!\nBiểu hiện của bạn có thể gặp do nhi...
3,Điều trị tiền đình bằng máy TRV chi phí bao nh...,Tôi bị rối loạn tiền điền thỉnh thoảng bị đau ...,"Chào anh,\nĐiều trị tiền đình do thạch nhĩ lạc..."
4,Trẻ bị tay chân miệng sốt mấy ngày thì khỏi? B...,"Xin chào bác sĩ, con trai tôi hiện đã được 2 t...",Trẻ bị tay chân miệng sốt mấy ngày thì khỏi bệ...


In [ ]:
a = [2, 1, 3, 5]
a.sort()
a

[1, 2, 3, 5]

In [ ]:
len(df)

3660

In [ ]:
df.to_csv('QA.tsv', index=False, sep='\t')

In [ ]:
df.to_csv('QA.csv', index=False,)